In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import re
from pyproj import Transformer
from kakao_map import lat_lon
warnings.filterwarnings('ignore')

## Data Load

- 서울시 초중고 정보: [출처](https://data.seoul.go.kr/dataList/OA-20502/S/1/datasetView.do)
- 서울시 대학교 정보: [출처](https://www.data.go.kr/data/15107736/standard.do)
- 서울시 극장 수 데이터: [출처](https://www.kobis.or.kr/kobis/business/mast/thea/findTheaterInfoList.do#none)
- 서울시 병원 위치 정보: [출처](https://data.seoul.go.kr/dataList/OA-20337/S/1/datasetView.do)
- 서울시 대규모 점포 정보: [출처](https://data.seoul.go.kr/dataList/OA-16096/S/1/datasetView.do)

In [3]:
school = pd.read_csv("../data/서울시-초중고.csv", encoding='euc-kr')
university = pd.read_csv("../data/서울시-대학교.csv", encoding='euc-kr')
movie = pd.read_csv("../data/서울시-극장수.csv", header=4,)
hospital = pd.read_csv("../data/서울시 병의원 위치 정보.csv", encoding='cp949')
mart = pd.read_csv("../data/서울시 대규모점포 인허가 정보.csv", encoding='cp949')
xy = pd.read_csv("../data/xy.csv")

print(f"School Data: {school.shape}")
print(f"University Data: {university.shape}")
print(f"Movie Data: {movie.shape}")
print(f"Hospital Data: {hospital.shape}")
print(f"Mart Data: {mart.shape}")
print(f"Xy Data: {xy.shape}")

School Data: (3935, 28)
University Data: (1961, 20)
Movie Data: (98, 21)
Hospital Data: (21485, 34)
Mart Data: (1012, 26)
Xy Data: (8167, 5)


### School Data

In [3]:
school.columns

Index(['학교종류명', '설립구분', '표준학교코드', '학교명', '영문학교명', '관할조직명', '도로명우편번호', '도로명주소',
       '도로명상세주소', '전화번호', '홈페이지주소', '팩스번호', '남녀공학구분명', '고등학교구분명',
       '산업체특별학급존재여부', '고등학교일반실업구분명', '특수목적고등학교계열명', '입시전후기구분명', '주야구분명',
       '설립일자', '개교기념일', '시도교육청코드', '시도교육청명', '소재지명', '주야과정', '계열명', '학과명',
       '적재일시'],
      dtype='object')

In [4]:
school = school[['도로명주소', '도로명상세주소', '학교명', '학교종류명']]
school['구'] = school['도로명주소'].apply(lambda x: x.split()[1])
school['주소키'] = school['도로명주소'].astype(str).str.strip() + ' ' + school['도로명상세주소'].astype(str).str.strip()
school['주소키'] = school['주소키'].str.replace(r'\([^)]*\)', '', regex=True)
school['주소키'] = school['주소키'].str.replace('/', ' ', regex=False) # / 제거
school['주소키'] = school['주소키'].str.replace('?', '', regex=False) # ? 제거
school = school[['구', '학교명', '학교종류명', '도로명주소', '주소키']]
school

,구,학교명,학교종류명,도로명주소,주소키
0,광진구,동국대학교사범대학부속가람중학교,중학교,서울특별시 광진구 광나루로36길 47,"서울특별시 광진구 광나루로36길 47 , 동국대학교사범대학부속가람중학교"
1,광진구,선화예술중학교,각종학교(중),서울특별시 광진구 천호대로 664,서울특별시 광진구 천호대로 664 선화예술중고등학교
2,성동구,서울숭신초등학교,초등학교,서울특별시 성동구 마장로 161,서울특별시 성동구 마장로 161
3,성동구,행당중학교,중학교,서울특별시 성동구 왕십리로 189,서울특별시 성동구 왕십리로 189
4,성동구,한양대학교사범대학부속중학교,중학교,서울특별시 성동구 마조로 42,서울특별시 성동구 마조로 42
...,...,...,...,...,...
3930,강남구,국립국악고등학교,고등학교,서울특별시 강남구 개포로22길 65,서울특별시 강남구 개포로22길 65
3931,마포구,한국우진학교,특수학교,서울특별시 마포구 월드컵북로38길 21,서울특별시 마포구 월드컵북로38길 21 한국우진학교
3932,종로구,서울농학교,특수학교,서울특별시 종로구 필운대로 103,서울특별시 종로구 필운대로 103
3933,종로구,서울맹학교,특수학교,서울특별시 종로구 필운대로 97,서울특별시 종로구 필운대로 97


In [5]:
school['학교종류명'].unique()

array(['중학교', '각종학교(중)', '초등학교', '각종학교(고)', '평생학교(초)-3년6학기', '각종학교(초)',
       '방송통신중학교', '고등학교', '특수학교', '외국인학교', '평생학교(고)-2년6학기',
       '평생학교(고)-3년6학기', '평생학교(중)-2년6학기', '고등기술학교', '방송통신고등학교', '공동실습소'],
      dtype=object)

In [6]:
school = school[school['학교종류명'].isin(['초등학교', '중학교', '고등학교'])]

print(school['학교종류명'].unique())
school

['중학교' '초등학교' '고등학교']


,구,학교명,학교종류명,도로명주소,주소키
0,광진구,동국대학교사범대학부속가람중학교,중학교,서울특별시 광진구 광나루로36길 47,"서울특별시 광진구 광나루로36길 47 , 동국대학교사범대학부속가람중학교"
2,성동구,서울숭신초등학교,초등학교,서울특별시 성동구 마장로 161,서울특별시 성동구 마장로 161
3,성동구,행당중학교,중학교,서울특별시 성동구 왕십리로 189,서울특별시 성동구 왕십리로 189
4,성동구,한양대학교사범대학부속중학교,중학교,서울특별시 성동구 마조로 42,서울특별시 성동구 마조로 42
5,광진구,자양중학교,중학교,서울특별시 광진구 뚝섬로41길 33,서울특별시 광진구 뚝섬로41길 33
...,...,...,...,...,...
3925,금천구,국립전통예술고등학교,고등학교,서울특별시 금천구 시흥대로38길 62,서울특별시 금천구 시흥대로38길 62
3926,금천구,국립전통예술고등학교,고등학교,서울특별시 금천구 시흥대로38길 62,서울특별시 금천구 시흥대로38길 62
3927,금천구,국립전통예술고등학교,고등학교,서울특별시 금천구 시흥대로38길 62,서울특별시 금천구 시흥대로38길 62
3929,강남구,국립국악고등학교,고등학교,서울특별시 강남구 개포로22길 65,서울특별시 강남구 개포로22길 65


In [7]:
university.columns

Index(['학교명', '학교 영문명', '본분교구분명', '대학구분명', '학교구분명', '설립형태구분명', '시도코드', '시도명',
       '소재지도로명주소', '소재지지번주소', '도로명우편번호', '소재지우편번호', '홈페이지주소', '대표전화번호',
       '대표팩스번호', '설립일자', '기준연도', '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')

In [8]:
university = university[['시도명', '학교명', '대학구분명', '소재지도로명주소']]
university

,시도명,학교명,대학구분명,소재지도로명주소
0,경기도,강남대학교 융복합대학원,대학원,"경기도 용인시 기흥구 강남로 40 (구갈동, 강남대학교)"
1,서울특별시,강서대학교 교육대학원,대학원,"서울특별시 강서구 까치산로24길 47 (화곡동, 케이씨대학교)"
2,서울특별시,강서대학교 사회복지대학원,대학원,"서울특별시 강서구 까치산로24길 47 (화곡동, 케이씨대학교)"
3,서울특별시,강서대학교 신학대학원,대학원,"서울특별시 강서구 까치산로24길 47 (화곡동, 케이씨대학교)"
4,서울특별시,강서대학교 일반대학원,대학원,"서울특별시 강서구 까치산로24길 47 (화곡동, 케이씨대학교)"
...,...,...,...,...
1956,서울특별시,국민대학교 행정대학원,대학원,"서울특별시 성북구 정릉로 77 (정릉동, 국민대학교)"
1957,충청북도,극동대학교 교육대학원,대학원,"충청북도 음성군 감곡면 대학길 76-32 (왕장리, 극동대학교)"
1958,충청북도,극동대학교 글로벌대학원,대학원,"충청북도 음성군 감곡면 대학길 76-32 (왕장리, 극동대학교)"
1959,충청북도,극동대학교 보건과학대학원,대학원,"충청북도 음성군 감곡면 대학길 76-32 (왕장리, 극동대학교)"


In [9]:
seoul_university = university[university['시도명'] == '서울특별시']
seoul_university

,시도명,학교명,대학구분명,소재지도로명주소
1,서울특별시,강서대학교 교육대학원,대학원,"서울특별시 강서구 까치산로24길 47 (화곡동, 케이씨대학교)"
2,서울특별시,강서대학교 사회복지대학원,대학원,"서울특별시 강서구 까치산로24길 47 (화곡동, 케이씨대학교)"
3,서울특별시,강서대학교 신학대학원,대학원,"서울특별시 강서구 까치산로24길 47 (화곡동, 케이씨대학교)"
4,서울특별시,강서대학교 일반대학원,대학원,"서울특별시 강서구 까치산로24길 47 (화곡동, 케이씨대학교)"
16,서울특별시,건국대학교 건축전문대학원,대학원,"서울특별시 광진구 능동로 120 (화양동, 건국대학교)"
...,...,...,...,...
1952,서울특별시,국민대학교 자동차산업대학원,대학원,"서울특별시 성북구 정릉로 77 (정릉동, 국민대학교)"
1953,서울특별시,국민대학교 정치대학원,대학원,"서울특별시 성북구 정릉로 77 (정릉동, 국민대학교)"
1954,서울특별시,국민대학교 종합예술대학원,대학원,"서울특별시 성북구 정릉로 77 (정릉동, 국민대학교)"
1955,서울특별시,국민대학교 테크노디자인전문대학원,대학원,"서울특별시 성북구 정릉로 77 (정릉동, 국민대학교)"


In [10]:
seoul_university = seoul_university[~seoul_university['대학구분명'].str.contains('대학원')]
seoul_university = seoul_university.drop_duplicates(subset='소재지도로명주소') # 주소 기준으로 중복 제거
seoul_university

,시도명,학교명,대학구분명,소재지도로명주소
85,서울특별시,한양여자대학교,전문대학,"서울특별시 성동구 살곶이길 200 (사근동, 한양여자대학)"
101,서울특별시,한국폴리텍 I 대학 서울강서캠퍼스,전문대학,"서울특별시 강서구 우장산로10길 112 (화곡동, 한국폴리텍Ⅰ서울강서대학)"
102,서울특별시,한국폴리텍 I 대학 서울정수캠퍼스,전문대학,"서울특별시 용산구 보광로 73 (보광동, 한국폴리텍1대학서울정수캠퍼스)"
122,서울특별시,적십자간호대학,전문대학,"서울특별시 동작구 흑석로 84 (흑석동, 중앙대학교)"
147,서울특별시,숭의여자대학교,전문대학,"서울특별시 중구 소파로2길 10 (예장동, 숭의여자대학)"
...,...,...,...,...
780,서울특별시,경희대학교,대학,"서울특별시 동대문구 경희대로 26 (회기동, 경희대학교)"
781,서울특별시,경희사이버대학교,대학,"서울특별시 동대문구 경희대로 26 (회기동, 경희사이버대학교)"
783,서울특별시,고려대학교,대학,"서울특별시 성북구 안암로 145 (안암동5가, 고려대학교안암캠퍼스(인문사회계))"
785,서울특별시,고려사이버대학교,대학,"서울특별시 종로구 북촌로 106 (계동, 고려사이버대학교)"


In [11]:
seoul_university = seoul_university[['학교명', '대학구분명', '소재지도로명주소']]
seoul_university['구'] = seoul_university['소재지도로명주소'].apply(lambda x: x.split()[1])
seoul_university = seoul_university[['구', '학교명', '대학구분명', '소재지도로명주소']]
seoul_university

,구,학교명,대학구분명,소재지도로명주소
85,성동구,한양여자대학교,전문대학,"서울특별시 성동구 살곶이길 200 (사근동, 한양여자대학)"
101,강서구,한국폴리텍 I 대학 서울강서캠퍼스,전문대학,"서울특별시 강서구 우장산로10길 112 (화곡동, 한국폴리텍Ⅰ서울강서대학)"
102,용산구,한국폴리텍 I 대학 서울정수캠퍼스,전문대학,"서울특별시 용산구 보광로 73 (보광동, 한국폴리텍1대학서울정수캠퍼스)"
122,동작구,적십자간호대학,전문대학,"서울특별시 동작구 흑석로 84 (흑석동, 중앙대학교)"
147,중구,숭의여자대학교,전문대학,"서울특별시 중구 소파로2길 10 (예장동, 숭의여자대학)"
...,...,...,...,...
780,동대문구,경희대학교,대학,"서울특별시 동대문구 경희대로 26 (회기동, 경희대학교)"
781,동대문구,경희사이버대학교,대학,"서울특별시 동대문구 경희대로 26 (회기동, 경희사이버대학교)"
783,성북구,고려대학교,대학,"서울특별시 성북구 안암로 145 (안암동5가, 고려대학교안암캠퍼스(인문사회계))"
785,종로구,고려사이버대학교,대학,"서울특별시 종로구 북촌로 106 (계동, 고려사이버대학교)"


In [12]:
seoul_university = seoul_university.rename(columns={'대학구분명':'학교구분명'})
school = school.rename(columns={'학교종류명':'학교구분명', '주소키':'소재지도로명주소'}).drop(columns=['도로명주소'])

seoul_school = pd.concat([seoul_university, school], ignore_index=True)
seoul_school = seoul_school.drop_duplicates(subset='소재지도로명주소').reset_index(drop=True)
seoul_school

,구,학교명,학교구분명,소재지도로명주소
0,성동구,한양여자대학교,전문대학,"서울특별시 성동구 살곶이길 200 (사근동, 한양여자대학)"
1,강서구,한국폴리텍 I 대학 서울강서캠퍼스,전문대학,"서울특별시 강서구 우장산로10길 112 (화곡동, 한국폴리텍Ⅰ서울강서대학)"
2,용산구,한국폴리텍 I 대학 서울정수캠퍼스,전문대학,"서울특별시 용산구 보광로 73 (보광동, 한국폴리텍1대학서울정수캠퍼스)"
3,동작구,적십자간호대학,전문대학,"서울특별시 동작구 흑석로 84 (흑석동, 중앙대학교)"
4,중구,숭의여자대학교,전문대학,"서울특별시 중구 소파로2길 10 (예장동, 숭의여자대학)"
...,...,...,...,...
1319,강남구,경기고등학교,고등학교,서울특별시 강남구 영동대로 643 경기고등학교
1320,강남구,개포고등학교,고등학교,서울특별시 강남구 개포로 402
1321,송파구,가락고등학교,고등학교,서울특별시 송파구 송이로 42
1322,금천구,국립전통예술고등학교,고등학교,서울특별시 금천구 시흥대로38길 62


In [ ]:
seoul_school['위도'] = None
seoul_school['경도'] = None

# 카카오 API 키
api_key = " "

# 주소 기반 위경도 수집
for i in range(len(seoul_school)):
    address = seoul_school.iloc[i]['소재지도로명주소']
    
    lat, lon = lat_lon(address, api_key)
    seoul_school.at[i, '위도'] = lat
    seoul_school.at[i, '경도'] = lon
    print(f"[{i}] 주소: {address} → 위도: {lat}, 경도: {lon}")
    time.sleep(0.3)  # 카카오 API는 초당 10건 제한

seoul_school

[0] 주소: 서울특별시 성동구 살곶이길 200 (사근동, 한양여자대학) → 위도: 37.5579175887117, 경도: 127.049321897316
[1] 주소: 서울특별시 강서구 우장산로10길 112 (화곡동, 한국폴리텍Ⅰ서울강서대학) → 위도: 37.5492424830708, 경도: 126.842445128444
[2] 주소: 서울특별시 용산구 보광로 73 (보광동, 한국폴리텍1대학서울정수캠퍼스) → 위도: 37.5299749717494, 경도: 126.996816950671
[3] 주소: 서울특별시 동작구 흑석로 84 (흑석동, 중앙대학교) → 위도: 37.5047267237807, 경도: 126.953833907628
[4] 주소: 서울특별시 중구 소파로2길 10 (예장동, 숭의여자대학) → 위도: 37.5578530747432, 경도: 126.987481605528
[5] 주소: 서울특별시 노원구 초안산로 12 (월계동, 인덕대학) → 위도: 37.6302514486494, 경도: 127.054448642206
[6] 주소: 서울특별시 동대문구 망우로 82 (휘경동, 삼육보건대학) → 위도: 37.5862119387672, 경도: 127.06355328121
[7] 주소: 서울특별시 서대문구 간호대로 38 (홍제동, 서울여자간호대학교) → 위도: 37.5976776713197, 경도: 126.947593444101
[8] 주소: 서울특별시 중랑구 용마산로90길 28 (면목동) → 위도: 37.5860827979004, 경도: 127.097203310115
[9] 주소: 서울특별시 구로구 경인로 445 (고척동, 동양미래대학) → 위도: 37.4999957879198, 경도: 126.868169656727
[10] 주소: 서울특별시 서대문구 가좌로 134 (홍은동, 명지전문대학) → 위도: 37.5852831941405, 경도: 126.925647822893
[11] 주소: 서울특별시 종로구 필운대로1길 34 (필운동, 배화여자대학) → 위도: 3

,구,학교명,학교구분명,소재지도로명주소,위도,경도
0,성동구,한양여자대학교,전문대학,"서울특별시 성동구 살곶이길 200 (사근동, 한양여자대학)",37.5579175887117,127.049321897316
1,강서구,한국폴리텍 I 대학 서울강서캠퍼스,전문대학,"서울특별시 강서구 우장산로10길 112 (화곡동, 한국폴리텍Ⅰ서울강서대학)",37.5492424830708,126.842445128444
2,용산구,한국폴리텍 I 대학 서울정수캠퍼스,전문대학,"서울특별시 용산구 보광로 73 (보광동, 한국폴리텍1대학서울정수캠퍼스)",37.5299749717494,126.996816950671
3,동작구,적십자간호대학,전문대학,"서울특별시 동작구 흑석로 84 (흑석동, 중앙대학교)",37.5047267237807,126.953833907628
4,중구,숭의여자대학교,전문대학,"서울특별시 중구 소파로2길 10 (예장동, 숭의여자대학)",37.5578530747432,126.987481605528
...,...,...,...,...,...,...
1319,강남구,경기고등학교,고등학교,서울특별시 강남구 영동대로 643 경기고등학교,37.517564539911,127.056075505663
1320,강남구,개포고등학교,고등학교,서울특별시 강남구 개포로 402,37.4850425995284,127.059305485497
1321,송파구,가락고등학교,고등학교,서울특별시 송파구 송이로 42,37.5010563298142,127.116425801136
1322,금천구,국립전통예술고등학교,고등학교,서울특별시 금천구 시흥대로38길 62,37.4463710473917,126.906462134788


In [6]:
seoul_school[seoul_school[['위도', '경도']].isna().any(axis=1)]

,구,학교명,학교구분명,소재지도로명주소,위도,경도
196,관악구,서울당곡초등학교,초등학교,서울특별시 관악구 보라매로2길 23 23,NaN,NaN
210,송파구,서울위례솔초등학교,초등학교,서울특별시 송파구 위례북로2길 12 12,NaN,NaN
646,용산구,서울신용산초등학교,초등학교,서울특별시 용산구 이촌로 255 301-75번지,NaN,NaN
1140,중랑구,이화여자대학교병설미디어고등학교,고등학교,서울특별시 중랑구 망우로73길 56 02061,NaN,NaN


In [7]:
seoul_school.dropna(subset=['위도', '경도'], inplace=True)
seoul_school.isna().sum()

구           0
학교명         0
학교구분명       0
소재지도로명주소    0
위도          0
경도          0
dtype: int64

In [8]:
seoul_school.to_csv("../data/School.csv", index=False)

### Movie Data

In [3]:
movie.head(3)

,광역단체,기초단체,영화상영관코드,영화상영관명,총 스크린수,총 좌석수,필름 상영관수,2D 상영관수,3D 상영관수,4D 상영관수,...,상설여부,특별관운영여부,가입여부,전송사업자명,개관일,영업상태,운영형태,주소,전화번호,홈페이지
0,서울시,강동구,1298,CGV 고덕강일,8,827,0,0,0,0,...,상설,NaN,가입,CJ올리브네트웍스(주),NaN,영업,직영,서울특별시 강동구 고덕동 353-23 번지 3층,02-371-6660,NaN
1,서울시,은평구,1106,CGV 불광,9,"1,600",0,9,3,0,...,상설,NaN,가입,CJ올리브네트웍스(주),2005-06-25,영업,위탁,서울특별시 은평구 대조동 240 번지 팜스퀘어 11층,1544-1122,http://www.cgv.co.kr/
2,서울시,강남구,1003,메가박스 코엑스,20,"3,431",0,20,3,0,...,상설,NaN,가입,메가박스(주),2000-05-13,영업,직영,서울특별시 강남구 삼성동 159 번지,1544-0070,http://www.megabox.co.kr/
3,서울시,강남구,1242,픽처하우스,1,59,0,1,0,0,...,상설,NaN,가입,메가박스(주),2017-01-11,영업,직영,서울특별시 강남구 신사동 532-1 번지 대원빌딩 B2,NaN,NaN
4,서울시,용산구,1088,CGV 용산아이파크몰,23,"5,247",0,22,7,1,...,상설,NaN,가입,CJ올리브네트웍스(주),2004-10-08,영업,직영,서울특별시 용산구 한강로3가 40-999 번지 아이파크몰 6층,1544-1122,http://www.cgv.co.kr/


In [4]:
movie = movie.drop(columns=['광역단체'])
movie = movie[['기초단체', '영화상영관명', '주소']]
movie = movie.rename(columns={'기초단체':'구'})
movie

,구,영화상영관명,주소
0,강동구,CGV 고덕강일,서울특별시 강동구 고덕동 353-23 번지 3층
1,은평구,CGV 불광,서울특별시 은평구 대조동 240 번지 팜스퀘어 11층
2,강남구,메가박스 코엑스,서울특별시 강남구 삼성동 159 번지
3,강남구,픽처하우스,서울특별시 강남구 신사동 532-1 번지 대원빌딩 B2
4,용산구,CGV 용산아이파크몰,서울특별시 용산구 한강로3가 40-999 번지 아이파크몰 6층
...,...,...,...
93,강서구,메가박스 마곡,서울특별시 강서구 마곡동 797-1 번지 퀸즈파크나인 B동 4층 메가박스 마곡지점
94,동대문구,롯데시네마 청량리,서울특별시 동대문구 전농동 591-53 번지
95,광진구,메가박스 군자,서울특별시 광진구 군자동 477-12 번지
96,강동구,메가박스강동,서울특별시 강동구 성내동 549-1 번지


In [ ]:
movie['위도'] = None
movie['경도'] = None

# 카카오 API 키
api_key = " "

# 주소 기반 위경도 수집
for i in range(len(movie)):
    address = movie.iloc[i]['주소']
    
    lat, lon = lat_lon(address, api_key)
    movie.at[i, '위도'] = lat
    movie.at[i, '경도'] = lon

    print(f"[{i:02}] {movie.iloc[i]['영화상영관명']} | 주소: {address} → 위도: {lat}, 경도: {lon}")
    time.sleep(0.3) # 카카오 API는 초당 10건 제한

movie

[00] CGV 고덕강일 | 주소: 서울특별시 강동구 고덕동 353-23 번지 3층 → 위도: 37.5662615261099, 경도: 127.157531058814
[01] CGV 불광 | 주소: 서울특별시 은평구 대조동 240 번지 팜스퀘어 11층 → 위도: 37.6097135228284, 경도: 126.928942235642
[02] 메가박스 코엑스 | 주소: 서울특별시 강남구 삼성동 159 번지 → 위도: 37.5125211887102, 경도: 127.058819250731
[03] 픽처하우스 | 주소: 서울특별시 강남구 신사동 532-1 번지 대원빌딩 B2 → 위도: 37.5230068645503, 경도: 127.022019199912
[04] CGV 용산아이파크몰 | 주소: 서울특별시 용산구 한강로3가 40-999 번지 아이파크몰 6층 → 위도: 37.5295804832797, 경도: 126.964026132479
[05] 메가박스송파파크하비오 | 주소: 서울특별시 송파구 문정동 618 번지 204동 지하1층 → 위도: 37.4806047633012, 경도: 127.12366621502
[06] 메가박스 동대문 | 주소: 서울특별시 중구 을지로6가 18-21 번지 굿모닝시티 9층~11층 → 위도: 37.5668732105026, 경도: 127.007280874922
[07] 모노플렉스앳라이즈 | 주소: 서울특별시 마포구 서교동 354-5 번지 4층 → 위도: 37.5544330596261, 경도: 126.921251202706
[08] CGV 강남 | 주소: 서울특별시 강남구 역삼동 814-6 번지 스타플렉스 4층~11층 → 위도: 37.5016524425806, 경도: 127.026374890126
[09] CGV 천호 | 주소: 서울특별시 강동구 천호동 42 번지 홈플러스 4층 → 위도: 37.5457178357509, 경도: 127.142250680744
[10] 메가박스 화곡 | 주소: 서울특별시 강서구 화곡동 1073-10 번지 메가스퀘어 4

,구,영화상영관명,주소,위도,경도
0,강동구,CGV 고덕강일,서울특별시 강동구 고덕동 353-23 번지 3층,37.5662615261099,127.157531058814
1,은평구,CGV 불광,서울특별시 은평구 대조동 240 번지 팜스퀘어 11층,37.6097135228284,126.928942235642
2,강남구,메가박스 코엑스,서울특별시 강남구 삼성동 159 번지,37.5125211887102,127.058819250731
3,강남구,픽처하우스,서울특별시 강남구 신사동 532-1 번지 대원빌딩 B2,37.5230068645503,127.022019199912
4,용산구,CGV 용산아이파크몰,서울특별시 용산구 한강로3가 40-999 번지 아이파크몰 6층,37.5295804832797,126.964026132479
...,...,...,...,...,...
93,강서구,메가박스 마곡,서울특별시 강서구 마곡동 797-1 번지 퀸즈파크나인 B동 4층 메가박스 마곡지점,37.5593146837116,126.835218262305
94,동대문구,롯데시네마 청량리,서울특별시 동대문구 전농동 591-53 번지,37.5809392592331,127.046949893617
95,광진구,메가박스 군자,서울특별시 광진구 군자동 477-12 번지,37.5557805929708,127.078340186484
96,강동구,메가박스강동,서울특별시 강동구 성내동 549-1 번지,37.5285308051505,127.125148250145


In [10]:
movie[movie[['위도', '경도']].isna().any(axis=1)]

,구,영화상영관명,주소,위도,경도
40,강서구,롯데시네마 김포공항,서울특별시 강서구 방화동 886 번지 외6필지,NaN,NaN
42,은평구,롯데시네마 은평(롯데몰),"서울특별시 은평구 진관동 79-31, 롯데몰 5층",NaN,NaN
44,강북구,롯데시네마 수유,서울특별시 강북구 번동 449-1 번지 10~14층,NaN,NaN
66,노원구,롯데시네마 수락산,서울특별시 노원구 상계동 1132-35 번지 3~9층,NaN,NaN
77,서초구,CGV Salon 서초,서울특별시 서초구 서초동 1335 번지 B2층,NaN,NaN


In [11]:
movie.dropna(subset=['위도', '경도'], inplace=True)
movie.isna().sum()

구         0
영화상영관명    0
주소        0
위도        0
경도        0
dtype: int64

In [12]:
movie.to_csv("../data/Movie.csv", index=False)

### Hospital Data

In [6]:
hospital.head(1)

,기관ID,주소,병원분류,병원분류명,응급의료기관코드,응급의료기관코드명,응급실운영여부(1/2),비고,기관설명상세,간이약도,...,진료시간(목요일)S,진료시간(금요일)S,진료시간(토요일)S,진료시간(일요일)S,진료시간(공휴일)S,우편번호1,우편번호2,병원경도,병원위도,작업시간
0,A1106598,"서울특별시 동작구 상도로 170, 2층 (상도동)",C,의원,G099,응급의료기관 이외,2,점심시간 12:30~13:30,NaN,장승배기역 3번출구 바로앞,...,900,900,900.0,NaN,NaN,69,56,126.938038,37.504076,2025-04-07 15:14:41.0


In [5]:
hospital.columns

Index(['기관ID', '주소', '병원분류', '병원분류명', '응급의료기관코드', '응급의료기관코드명', '응급실운영여부(1/2)',
       '비고', '기관설명상세', '간이약도', '기관명', '대표전화1', '응급실전화', '진료시간(월요일)C',
       '진료시간(화요일)C', '진료시간(수요일)C', '진료시간(목요일)C', '진료시간(금요일)C', '진료시간(토요일)C',
       '진료시간(일요일)C', '진료시간(공휴일)C', '진료시간(월요일)S', '진료시간(화요일)S', '진료시간(수요일)S',
       '진료시간(목요일)S', '진료시간(금요일)S', '진료시간(토요일)S', '진료시간(일요일)S', '진료시간(공휴일)S',
       '우편번호1', '우편번호2', '병원경도', '병원위도', '작업시간'],
      dtype='object')

In [7]:
filter_cols = ['주소', '기관명', '병원분류명', '병원위도', '병원경도']
hospital = hospital[filter_cols]
hospital

,주소,기관명,병원분류명,병원위도,병원경도
0,"서울특별시 동작구 상도로 170, 2층 (상도동)",(사)한국한센복지협회서울특별시지부부설의원,의원,37.504076,126.938038
1,서울특별시 강남구 봉은사로 612 (삼성동),(의)가산의료재단광동병원,병원,37.514281,127.062147
2,"서울특별시 마포구 월드컵로1길 14, 2층 212~215호 (합정동, 마포 한강 푸...",365더잘봄정형외과의원,의원,37.550050,126.912061
3,"서울특별시 강동구 양재대로 1360, 포레온스테이션5 3층 3004~3010, 30...",365웰정형외과의원,의원,37.527476,127.137046
4,"서울특별시 강남구 테헤란로 322, 한신인터밸리24 지하2층 B102, B103호 ...",가뿐한의원,한의원,37.503236,127.046582
...,...,...,...,...,...
21480,서울특별시 강서구 곰달래로 223 (화곡동),힘찬부성정형외과의원,의원,37.531808,126.857787
21481,"서울특별시 용산구 한강대로 109 (한강로2가, 용성비즈텔)",힘찬세상경희한의원,한의원,37.529991,126.968173
21482,"서울특별시 강남구 봉은사로 213, 센트럴타워 5~7층 (논현동)",힘찬큐한방병원,한방병원,37.508050,127.035366
21483,"서울특별시 강서구 강서로 173, 터보빌딩 7,8,9층 (화곡동)",힘찬하나로한방병원,한방병원,37.541922,126.839622


In [10]:
hospital['구'] = hospital['주소'].apply(lambda x: x.split()[1])
order_col = ['구', '기관명', '병원분류명', '주소', '병원위도', '병원경도']
hospital = hospital[order_col]
hospital

,구,기관명,병원분류명,주소,병원위도,병원경도
0,동작구,(사)한국한센복지협회서울특별시지부부설의원,의원,"서울특별시 동작구 상도로 170, 2층 (상도동)",37.504076,126.938038
1,강남구,(의)가산의료재단광동병원,병원,서울특별시 강남구 봉은사로 612 (삼성동),37.514281,127.062147
2,마포구,365더잘봄정형외과의원,의원,"서울특별시 마포구 월드컵로1길 14, 2층 212~215호 (합정동, 마포 한강 푸...",37.550050,126.912061
3,강동구,365웰정형외과의원,의원,"서울특별시 강동구 양재대로 1360, 포레온스테이션5 3층 3004~3010, 30...",37.527476,127.137046
4,강남구,가뿐한의원,한의원,"서울특별시 강남구 테헤란로 322, 한신인터밸리24 지하2층 B102, B103호 ...",37.503236,127.046582
...,...,...,...,...,...,...
21480,강서구,힘찬부성정형외과의원,의원,서울특별시 강서구 곰달래로 223 (화곡동),37.531808,126.857787
21481,용산구,힘찬세상경희한의원,한의원,"서울특별시 용산구 한강대로 109 (한강로2가, 용성비즈텔)",37.529991,126.968173
21482,강남구,힘찬큐한방병원,한방병원,"서울특별시 강남구 봉은사로 213, 센트럴타워 5~7층 (논현동)",37.508050,127.035366
21483,강서구,힘찬하나로한방병원,한방병원,"서울특별시 강서구 강서로 173, 터보빌딩 7,8,9층 (화곡동)",37.541922,126.839622


In [12]:
print(hospital['구'].nunique())
hospital['구'].unique()

25


array(['동작구', '강남구', '마포구', '강동구', '영등포구', '금천구', '서초구', '은평구', '관악구',
       '양천구', '구로구', '송파구', '동대문구', '강서구', '서대문구', '노원구', '중구', '종로구',
       '용산구', '도봉구', '광진구', '중랑구', '강북구', '성북구', '성동구'], dtype=object)

In [13]:
hospital.isna().sum()

구        0
기관명      0
병원분류명    0
주소       0
병원위도     0
병원경도     0
dtype: int64

In [19]:
hospital.to_csv("../data/Hospital.csv", index=False)

### Mart Data

In [3]:
mart.columns

Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '점포구분명'],
      dtype='object')

In [4]:
filter_cols = ['사업장명', '폐업일자', '도로명주소', '지번주소', '좌표정보(X)', '좌표정보(Y)']
mart = mart[filter_cols]
mart

,사업장명,폐업일자,도로명주소,지번주소,좌표정보(X),좌표정보(Y)
0,신동아쇼핑센터,NaN,"서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)",서울특별시 용산구 용산동6가 69-167 신동아쇼핑,198622.892650,446114.155239
1,GS THE FRESH 둔촌포레온점,NaN,NaN,서울특별시 강동구 둔촌동 170-1,212195.109947,447197.273287
2,하나로마트 둔촌점,NaN,NaN,서울특별시 강동구 둔촌동 170-1,212195.109947,447197.273287
3,(주)이마트에브리데이 둔촌점,NaN,NaN,"서울특별시 강동구 둔촌동 174-1 B102,B103호",212027.481159,446598.591776
4,구의 자이르네 판매시설,NaN,서울특별시 광진구 광나루로39길 11 (구의동),서울특별시 광진구 구의동 66-25,207775.865132,449252.386906
...,...,...,...,...,...,...
1007,GS THE FRESH 강동성내점,NaN,"서울특별시 강동구 풍성로38길 9, 바로빌딩 1층 (성내동)",서울특별시 강동구 성내동 293-12 바로빌딩,211027.511753,447708.220614
1008,천호코오롱상가,NaN,서울특별시 강동구 진황도로 12 (천호동),서울특별시 강동구 천호동 410번지 100호,211279.030242,448548.819717
1009,신세계 이마트 명일점,NaN,NaN,서울특별시 강동구 명일동 46번지 4호,213700.877715,450278.100931
1010,양지종합시장,NaN,NaN,서울특별시 강동구 암사3동 451번지 16호,212082.014198,450348.793644


In [5]:
mart.isna().sum()

사업장명         0
폐업일자       822
도로명주소      176
지번주소       117
좌표정보(X)     82
좌표정보(Y)     82
dtype: int64

In [6]:
# 폐업하지 않은 사업장
Not_closed_mart = mart[mart['폐업일자'].isna()].copy() 
Not_closed_mart = Not_closed_mart.dropna(subset=['도로명주소', '지번주소'], how='all').reset_index(drop=True)
Not_closed_mart

,사업장명,폐업일자,도로명주소,지번주소,좌표정보(X),좌표정보(Y)
0,신동아쇼핑센터,NaN,"서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)",서울특별시 용산구 용산동6가 69-167 신동아쇼핑,198622.892650,446114.155239
1,GS THE FRESH 둔촌포레온점,NaN,NaN,서울특별시 강동구 둔촌동 170-1,212195.109947,447197.273287
2,하나로마트 둔촌점,NaN,NaN,서울특별시 강동구 둔촌동 170-1,212195.109947,447197.273287
3,(주)이마트에브리데이 둔촌점,NaN,NaN,"서울특별시 강동구 둔촌동 174-1 B102,B103호",212027.481159,446598.591776
4,구의 자이르네 판매시설,NaN,서울특별시 광진구 광나루로39길 11 (구의동),서울특별시 광진구 구의동 66-25,207775.865132,449252.386906
...,...,...,...,...,...,...
817,GS THE FRESH 강동성내점,NaN,"서울특별시 강동구 풍성로38길 9, 바로빌딩 1층 (성내동)",서울특별시 강동구 성내동 293-12 바로빌딩,211027.511753,447708.220614
818,천호코오롱상가,NaN,서울특별시 강동구 진황도로 12 (천호동),서울특별시 강동구 천호동 410번지 100호,211279.030242,448548.819717
819,신세계 이마트 명일점,NaN,NaN,서울특별시 강동구 명일동 46번지 4호,213700.877715,450278.100931
820,양지종합시장,NaN,NaN,서울특별시 강동구 암사3동 451번지 16호,212082.014198,450348.793644


In [7]:
Not_closed_mart['지번주소'] = Not_closed_mart['지번주소'].apply(lambda x: x.strip() if isinstance(x, str) else x) # 앞뒤 공백 제거
Not_closed_mart[Not_closed_mart['지번주소'] == '호']

,사업장명,폐업일자,도로명주소,지번주소,좌표정보(X),좌표정보(Y)
36,종로세운상가,NaN,NaN,호,NaN,NaN
54,광장시장주식회사,NaN,NaN,호,NaN,NaN
220,기타대규모점포,NaN,NaN,호,NaN,NaN
306,강북종합시장,NaN,NaN,호,NaN,NaN
663,관악종합시장,NaN,NaN,호,NaN,NaN


In [8]:
Not_closed_mart = Not_closed_mart[Not_closed_mart['지번주소'] != '호']
Not_closed_mart

,사업장명,폐업일자,도로명주소,지번주소,좌표정보(X),좌표정보(Y)
0,신동아쇼핑센터,NaN,"서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)",서울특별시 용산구 용산동6가 69-167 신동아쇼핑,198622.892650,446114.155239
1,GS THE FRESH 둔촌포레온점,NaN,NaN,서울특별시 강동구 둔촌동 170-1,212195.109947,447197.273287
2,하나로마트 둔촌점,NaN,NaN,서울특별시 강동구 둔촌동 170-1,212195.109947,447197.273287
3,(주)이마트에브리데이 둔촌점,NaN,NaN,"서울특별시 강동구 둔촌동 174-1 B102,B103호",212027.481159,446598.591776
4,구의 자이르네 판매시설,NaN,서울특별시 광진구 광나루로39길 11 (구의동),서울특별시 광진구 구의동 66-25,207775.865132,449252.386906
...,...,...,...,...,...,...
817,GS THE FRESH 강동성내점,NaN,"서울특별시 강동구 풍성로38길 9, 바로빌딩 1층 (성내동)",서울특별시 강동구 성내동 293-12 바로빌딩,211027.511753,447708.220614
818,천호코오롱상가,NaN,서울특별시 강동구 진황도로 12 (천호동),서울특별시 강동구 천호동 410번지 100호,211279.030242,448548.819717
819,신세계 이마트 명일점,NaN,NaN,서울특별시 강동구 명일동 46번지 4호,213700.877715,450278.100931
820,양지종합시장,NaN,NaN,서울특별시 강동구 암사3동 451번지 16호,212082.014198,450348.793644


In [9]:
# 도로명주소 유효성 체크
valid_road = Not_closed_mart['도로명주소'].apply(
    lambda x: isinstance(x, str) and len(x.split()) > 1 and x.split()[1].endswith('구'))

# 지번주소 정제 컬럼 생성 (구 포함된 경우 앞 4개만 추출)
Not_closed_mart['지번주소_정제'] = Not_closed_mart['지번주소'].apply(
    lambda x: ' '.join(x.split()[:4]) if isinstance(x, str) and '구' in x else np.nan)

# 검색주소 생성: 도로명주소 유효하면 그대로, 아니면 지번주소_정제 사용
Not_closed_mart['검색주소'] = Not_closed_mart.apply(
    lambda row: row['도로명주소'] if valid_road[row.name]
    else row['지번주소_정제'],
    axis=1)

# '구' 컬럼 생성
Not_closed_mart['구'] = Not_closed_mart['검색주소'].apply(
    lambda x: x.split()[1] if isinstance(x, str) and len(x.split()) > 1 and x.split()[1].endswith('구') else np.nan)

Not_closed_mart = Not_closed_mart[['구', '사업장명', '검색주소', '도로명주소', '지번주소_정제', '지번주소', '좌표정보(X)', '좌표정보(Y)']]
Not_closed_mart

,구,사업장명,검색주소,도로명주소,지번주소_정제,지번주소,좌표정보(X),좌표정보(Y)
0,용산구,신동아쇼핑센터,"서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)","서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)",서울특별시 용산구 용산동6가 69-167,서울특별시 용산구 용산동6가 69-167 신동아쇼핑,198622.892650,446114.155239
1,강동구,GS THE FRESH 둔촌포레온점,서울특별시 강동구 둔촌동 170-1,NaN,서울특별시 강동구 둔촌동 170-1,서울특별시 강동구 둔촌동 170-1,212195.109947,447197.273287
2,강동구,하나로마트 둔촌점,서울특별시 강동구 둔촌동 170-1,NaN,서울특별시 강동구 둔촌동 170-1,서울특별시 강동구 둔촌동 170-1,212195.109947,447197.273287
3,강동구,(주)이마트에브리데이 둔촌점,서울특별시 강동구 둔촌동 174-1,NaN,서울특별시 강동구 둔촌동 174-1,"서울특별시 강동구 둔촌동 174-1 B102,B103호",212027.481159,446598.591776
4,광진구,구의 자이르네 판매시설,서울특별시 광진구 광나루로39길 11 (구의동),서울특별시 광진구 광나루로39길 11 (구의동),서울특별시 광진구 구의동 66-25,서울특별시 광진구 구의동 66-25,207775.865132,449252.386906
...,...,...,...,...,...,...,...,...
817,강동구,GS THE FRESH 강동성내점,"서울특별시 강동구 풍성로38길 9, 바로빌딩 1층 (성내동)","서울특별시 강동구 풍성로38길 9, 바로빌딩 1층 (성내동)",서울특별시 강동구 성내동 293-12,서울특별시 강동구 성내동 293-12 바로빌딩,211027.511753,447708.220614
818,강동구,천호코오롱상가,서울특별시 강동구 진황도로 12 (천호동),서울특별시 강동구 진황도로 12 (천호동),서울특별시 강동구 천호동 410번지,서울특별시 강동구 천호동 410번지 100호,211279.030242,448548.819717
819,강동구,신세계 이마트 명일점,서울특별시 강동구 명일동 46번지,NaN,서울특별시 강동구 명일동 46번지,서울특별시 강동구 명일동 46번지 4호,213700.877715,450278.100931
820,강동구,양지종합시장,서울특별시 강동구 암사3동 451번지,NaN,서울특별시 강동구 암사3동 451번지,서울특별시 강동구 암사3동 451번지 16호,212082.014198,450348.793644


In [10]:
Not_closed_mart.isna().sum()

구            0
사업장명         0
검색주소         0
도로명주소      135
지번주소_정제     86
지번주소        86
좌표정보(X)     64
좌표정보(Y)     64
dtype: int64

In [11]:
transformer = Transformer.from_crs("epsg:5178", "epsg:4326", always_xy=True)

# 변환 
Not_closed_mart['위도'], Not_closed_mart['경도'] = transformer.transform(
    Not_closed_mart['좌표정보(X)'].values,  # X = 경도
    Not_closed_mart['좌표정보(Y)'].values   # Y = 위도
)

# 컬럼 순서 정리
order_col = ['구', '사업장명', '검색주소', '도로명주소', '지번주소_정제', '지번주소', '위도', '경도']
Not_closed_mart = Not_closed_mart[order_col]
Not_closed_mart

,구,사업장명,검색주소,도로명주소,지번주소_정제,지번주소,위도,경도
0,용산구,신동아쇼핑센터,"서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)","서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)",서울특별시 용산구 용산동6가 69-167,서울특별시 용산구 용산동6가 69-167 신동아쇼핑,119.648868,23.781272
1,강동구,GS THE FRESH 둔촌포레온점,서울특별시 강동구 둔촌동 170-1,NaN,서울특별시 강동구 둔촌동 170-1,서울특별시 강동구 둔촌동 170-1,119.780278,23.797661
2,강동구,하나로마트 둔촌점,서울특별시 강동구 둔촌동 170-1,NaN,서울특별시 강동구 둔촌동 170-1,서울특별시 강동구 둔촌동 170-1,119.780278,23.797661
3,강동구,(주)이마트에브리데이 둔촌점,서울특별시 강동구 둔촌동 174-1,NaN,서울특별시 강동구 둔촌동 174-1,"서울특별시 강동구 둔촌동 174-1 B102,B103호",119.778966,23.792221
4,광진구,구의 자이르네 판매시설,서울특별시 광진구 광나루로39길 11 (구의동),서울특별시 광진구 광나루로39길 11 (구의동),서울특별시 광진구 구의동 66-25,서울특별시 광진구 구의동 66-25,119.736194,23.813885
...,...,...,...,...,...,...,...,...
817,강동구,GS THE FRESH 강동성내점,"서울특별시 강동구 풍성로38길 9, 바로빌딩 1층 (성내동)","서울특별시 강동구 풍성로38길 9, 바로빌딩 1층 (성내동)",서울특별시 강동구 성내동 293-12,서울특별시 강동구 성내동 293-12 바로빌딩,119.768648,23.801662
818,강동구,천호코오롱상가,서울특별시 강동구 진황도로 12 (천호동),서울특별시 강동구 진황도로 12 (천호동),서울특별시 강동구 천호동 410번지,서울특별시 강동구 천호동 410번지 100호,119.770647,23.809309
819,강동구,신세계 이마트 명일점,서울특별시 강동구 명일동 46번지,NaN,서울특별시 강동구 명일동 46번지,서울특별시 강동구 명일동 46번지 4호,119.793288,23.825973
820,강동구,양지종합시장,서울특별시 강동구 암사3동 451번지,NaN,서울특별시 강동구 암사3동 451번지,서울특별시 강동구 암사3동 451번지 16호,119.777500,23.825813


In [12]:
Not_closed_mart.isna().sum()

구            0
사업장명         0
검색주소         0
도로명주소      135
지번주소_정제     86
지번주소        86
위도          64
경도          64
dtype: int64

In [13]:
print(f"자치구 수: {Not_closed_mart['구'].nunique()}")
Not_closed_mart['구'].unique()

자치구 수: 25


array(['용산구', '강동구', '광진구', '동작구', '은평구', '성동구', '송파구', '서대문구', '서초구',
       '강북구', '강남구', '종로구', '중구', '동대문구', '중랑구', '성북구', '도봉구', '노원구',
       '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '관악구'], dtype=object)

In [14]:
drop_col = ['도로명주소', '지번주소_정제', '지번주소', '위도', '경도'] # 확인해보니 위도, 경도가 잘못 돼 있음
Not_closed_mart = Not_closed_mart.drop(columns=drop_col)
Not_closed_mart

,구,사업장명,검색주소
0,용산구,신동아쇼핑센터,"서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)"
1,강동구,GS THE FRESH 둔촌포레온점,서울특별시 강동구 둔촌동 170-1
2,강동구,하나로마트 둔촌점,서울특별시 강동구 둔촌동 170-1
3,강동구,(주)이마트에브리데이 둔촌점,서울특별시 강동구 둔촌동 174-1
4,광진구,구의 자이르네 판매시설,서울특별시 광진구 광나루로39길 11 (구의동)
...,...,...,...
817,강동구,GS THE FRESH 강동성내점,"서울특별시 강동구 풍성로38길 9, 바로빌딩 1층 (성내동)"
818,강동구,천호코오롱상가,서울특별시 강동구 진황도로 12 (천호동)
819,강동구,신세계 이마트 명일점,서울특별시 강동구 명일동 46번지
820,강동구,양지종합시장,서울특별시 강동구 암사3동 451번지


In [15]:
# 중복 제거
Not_closed_mart.drop_duplicates(subset=['검색주소'], inplace=True)
Not_closed_mart

,구,사업장명,검색주소
0,용산구,신동아쇼핑센터,"서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)"
1,강동구,GS THE FRESH 둔촌포레온점,서울특별시 강동구 둔촌동 170-1
3,강동구,(주)이마트에브리데이 둔촌점,서울특별시 강동구 둔촌동 174-1
4,광진구,구의 자이르네 판매시설,서울특별시 광진구 광나루로39길 11 (구의동)
5,동작구,GS THE FRESH 노량진역점,"서울특별시 동작구 만양로 93, 주영빌딩 B1층 (노량진동)"
...,...,...,...
817,강동구,GS THE FRESH 강동성내점,"서울특별시 강동구 풍성로38길 9, 바로빌딩 1층 (성내동)"
818,강동구,천호코오롱상가,서울특별시 강동구 진황도로 12 (천호동)
819,강동구,신세계 이마트 명일점,서울특별시 강동구 명일동 46번지
820,강동구,양지종합시장,서울특별시 강동구 암사3동 451번지


In [ ]:
Not_closed_mart['위도'] = None
Not_closed_mart['경도'] = None

# 카카오 API 키
api_key = " "

# 위경도 수집
for i in range(len(Not_closed_mart)):
    base_addr = Not_closed_mart.iloc[i]['검색주소']
    name = Not_closed_mart.iloc[i]['사업장명']

    # 1차 시도: 검색주소 + 사업장명
    full_addr = f"{base_addr} {name}" if pd.notna(base_addr) and pd.notna(name) else base_addr
    lat, lon = lat_lon(full_addr, api_key)

    # 2차 시도: 검색주소만
    if lat is None or lon is None and pd.notna(base_addr):
        lat, lon = lat_lon(base_addr, api_key)

    Not_closed_mart.at[i, '위도'] = lat
    Not_closed_mart.at[i, '경도'] = lon

    print(f"[{i}] 주소: {full_addr} → 위도: {lat}, 경도: {lon}")
    time.sleep(0.3)  # API 초당 10건 제한 고려

Not_closed_mart

[0] 주소: 서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가) 신동아쇼핑센터 → 위도: 37.5173837456591, 경도: 126.985175715456
[1] 주소: 서울특별시 강동구 둔촌동 170-1 GS THE FRESH 둔촌포레온점 → 위도: 37.5268477108188, 경도: 127.138949568286
[2] 주소: 서울특별시 강동구 둔촌동 174-1 (주)이마트에브리데이 둔촌점 → 위도: 37.5216453023545, 경도: 127.136420009953
[3] 주소: 서울특별시 광진구 광나루로39길 11 (구의동) 구의 자이르네 판매시설 → 위도: 37.5455028168806, 경도: 127.088780729862
[4] 주소: 서울특별시 동작구 만양로 93, 주영빌딩 B1층 (노량진동) GS THE FRESH 노량진역점 → 위도: 37.5118036816925, 경도: 126.944071659172
[5] 주소: 서울특별시 광진구 능동로13길 39 (화양동) (주)지에스리테일GS수퍼 광진화양점 → 위도: 37.543959020957, 경도: 127.069757721395
[6] 주소: 서울특별시 용산구 백범로 341 (원효로1가) 롯데쇼핑(주)롯데슈퍼원효로점 → 위도: 37.5379386735674, 경도: 126.967973433167
[7] 주소: 서울특별시 은평구 통일로 842 (불광동) 롯데쇼핑(주)롯데슈퍼범서점 → 위도: 37.6186902335511, 경도: 126.922063807625
[8] 주소: 서울특별시 성동구 마장로 137, 221동 1층 1230호,1231호,1232(일부)호 (상왕십리동, 텐즈힐) GS THE FRESH 성동텐즈힐점 → 위도: 37.5678250213489, 경도: 127.025053595952
[9] 주소: 서울특별시 용산구 이촌로 200, LG프라자 지하1층 (이촌동) GS THE FRESH 용산이촌점 → 위도: 37.5209511026518, 경도: 126.9693432407

,구,사업장명,검색주소,위도,경도
0,용산구,신동아쇼핑센터,"서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)",37.5173837456591,126.985175715456
1,강동구,GS THE FRESH 둔촌포레온점,서울특별시 강동구 둔촌동 170-1,37.5268477108188,127.138949568286
3,강동구,(주)이마트에브리데이 둔촌점,서울특별시 강동구 둔촌동 174-1,37.5455028168806,127.088780729862
4,광진구,구의 자이르네 판매시설,서울특별시 광진구 광나루로39길 11 (구의동),37.5118036816925,126.944071659172
5,동작구,GS THE FRESH 노량진역점,"서울특별시 동작구 만양로 93, 주영빌딩 B1층 (노량진동)",37.543959020957,127.069757721395
...,...,...,...,...,...
655,NaN,NaN,NaN,37.5233111622682,127.044119009562
663,NaN,NaN,NaN,37.4913209890296,127.088314234699
666,NaN,NaN,NaN,37.5086151302204,127.059765116537
689,NaN,NaN,NaN,37.4776148442516,127.124961768886


In [17]:
Not_closed_mart[Not_closed_mart[['위도', '경도']].isna().any(axis=1)]

,구,사업장명,검색주소,위도,경도
43,종로구,세운스퀘어,서울특별시 종로구 창경궁로 109 (인의동),None,None
49,종로구,신설종합시장(주),"서울특별시 종로구 난계로27길 51, 신설종합시장 (숭인동)",None,None
50,종로구,GS THE FRESH 무악현대점,"서울특별시 종로구 통일로 246-10, 무악현대아파트 지하 1층 (무악동)",None,None
64,중구,대림상가,서울특별시 중구 을지로 157 (을지로4가),None,None
78,중구,롯데아울렛 서울역점,서울특별시 중구 한강대로 405 (봉래동2가),None,None
...,...,...,...,...,...
454,NaN,NaN,NaN,None,None
477,NaN,NaN,NaN,None,None
517,NaN,NaN,NaN,None,None
591,NaN,NaN,NaN,None,None


In [20]:
Not_closed_mart = Not_closed_mart.dropna()
Not_closed_mart

,구,사업장명,검색주소,위도,경도
0,용산구,신동아쇼핑센터,"서울특별시 용산구 이촌로 352, 신동아쇼핑 (용산동6가)",37.5173837456591,126.985175715456
1,강동구,GS THE FRESH 둔촌포레온점,서울특별시 강동구 둔촌동 170-1,37.5268477108188,127.138949568286
3,강동구,(주)이마트에브리데이 둔촌점,서울특별시 강동구 둔촌동 174-1,37.5455028168806,127.088780729862
4,광진구,구의 자이르네 판매시설,서울특별시 광진구 광나루로39길 11 (구의동),37.5118036816925,126.944071659172
5,동작구,GS THE FRESH 노량진역점,"서울특별시 동작구 만양로 93, 주영빌딩 B1층 (노량진동)",37.543959020957,127.069757721395
...,...,...,...,...,...
734,강남구,진솔물산,서울특별시 강남구 역삼동 826번지,37.5662615261099,127.157531058814
735,강남구,(주)현대백화점 본점,"서울특별시 강남구 압구정로 165 (압구정동, 현대백화점본점)",37.5316068000518,127.12559371975
737,강남구,갤러리아백화점(서관),"서울특별시 강남구 압구정로 343 (압구정동, 갤러리아백화점)",37.5546138640691,127.153661406004
738,강남구,홈플러스 익스프레스 압구정점,서울특별시 강남구 압구정로46길 5-2 (신사동),37.5554424709589,127.136615593311


In [21]:
Not_closed_mart.isna().sum()

구       0
사업장명    0
검색주소    0
위도      0
경도      0
dtype: int64

In [23]:
Not_closed_mart.to_csv("../data/Mart.csv", index=False)

### Xy Data

In [3]:
xy.head()

,addr,x,y
0,서울특별시 강남구 역삼동 762,127.046917,37.495583
1,서울특별시 강북구 수유동 282-20,127.014052,37.646608
2,서울특별시 구로구 구로동 107-4,126.891127,37.498664
3,서울특별시 양천구 신정동 911-4,126.857998,37.526901
4,서울특별시 강서구 등촌동 676,126.860381,37.549660


In [10]:
xy['구'] = xy['addr'].apply(lambda X: X.split()[1])
xy['동'] = xy['addr'].apply(lambda X: X.split()[2])
xy = xy.rename(columns={'y':'위도', 'x':'경도'})

xy = xy[['구', '동', 'addr', '위도', '경도']]
xy

,구,동,addr,위도,경도
0,강남구,역삼동,서울특별시 강남구 역삼동 762,37.495583,127.046917
1,강북구,수유동,서울특별시 강북구 수유동 282-20,37.646608,127.014052
2,구로구,구로동,서울특별시 구로구 구로동 107-4,37.498664,126.891127
3,양천구,신정동,서울특별시 양천구 신정동 911-4,37.526901,126.857998
4,강서구,등촌동,서울특별시 강서구 등촌동 676,37.549660,126.860381
...,...,...,...,...,...
8162,서초구,서초동,서울특별시 서초구 서초동 1511-3,37.487429,127.007728
8163,강북구,수유동,서울특별시 강북구 수유동 130-43,37.637450,127.018409
8164,구로구,궁동,서울특별시 구로구 궁동 182,37.497958,126.830796
8165,강동구,성내동,서울특별시 강동구 성내동 419-13,37.528854,127.132973


In [11]:
xy.isna().sum()

구       0
동       0
addr    0
위도      0
경도      0
dtype: int64

In [ ]:
xy.to_csv("../data/xy.csv", index=False)